In [70]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Ruta del archivo CSV
ruta_csv = r'C:\Users\marib\OneDrive\Escritorio\295\data_ioc_1min_2023_2024\pich_prs.csv'
# Lectura del archivo CSV en un DataFrame
csv = pd.read_csv(ruta_csv)

csv['fecha'] = pd.to_datetime(csv['fecha']) # Asegurarse de que la fecha este en formato DateTime

## Filtro temporal y de outliers de los datos

In [73]:
# Filtrar los datos desde julio de 2023 hasta la actualidad
start_date = '2023-07-01 00:01:00'
end_date = pd.to_datetime('2024-09-30 23:59:00')
csv_filtered = csv[(csv['fecha'] >= start_date) & (csv['fecha'] <= end_date)]

In [75]:
# Limpieza data de outliers problematicos
rango_min_prs = 1.0  # valor mínimo permitido
rango_max_prs = 4.0  # valor máximo permitido

# Reemplazar los valores fuera del rango con NaN
csv_filtered['prs(m)'] = csv_filtered['prs(m)'].apply(lambda x: np.nan if not rango_min_prs <= x <= rango_max_prs else x)

C:\Users\marib\AppData\Local\Temp\ipykernel_12888\395208371.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_filtered['prs(m)'] = csv_filtered['prs(m)'].apply(lambda x: np.nan if not rango_min_prs <= x <= rango_max_prs else x)


In [77]:
%matplotlib qt 
# Graficar los datos del archivo CSV
#plt.xlabel('Fecha', fontsize=20) # Label de x
plt.ylabel('Nivel del Mar [m]', fontsize=20) # Label de y
plt.plot(csv_filtered['fecha'], csv_filtered['prs(m)'], color='black', label='NM[m]') # Ploteo de datos
plt.xticks(fontsize=18)                 
plt.yticks(fontsize=18) 

# Añadir leyenda
plt.tight_layout()
plt.legend(loc='upper right', fontsize=20)

# Mostrar el gráfico
plt.title('Sea Level Pichidangui (july 2023-september 2024)', fontsize=20) # titulo
plt.show() # Mostrar figura

## Cálculo de armonicos gravitacionales

In [79]:
# Utide
import utide
# Definimos los vectores de tiempo y nivel del mar 
t=csv_filtered['fecha']
u=csv_filtered['prs(m)']-csv['prs(m)'].mean() # Nivel del mar al rededor del promedio
# Cálculo de los coeficientes armónicos de la marea
coef = utide.solve(t,u,
    lat=-32.7756,
    method="ols",
    conf_int="MC",
    verbose=False)

In [80]:
# Creación de tabla de coeficientes
nombres = coef.get('name') # Nombre del armónico
amplitud = coef.get('A') # Amplitud del armónico
porcentaje_de_energia = coef.get('PE') # Porcentaje de energía del armónico
fase = coef.get('g') # Fase del armónico

# Crear un DataFrame con pandas
df_armonicos = pd.DataFrame({
    'Armonico': nombres,
    'Amplitud': amplitud,
    'Fase': fase,
    'Porcentaje de Energia': porcentaje_de_energia
})

# Mostrar la tabla
print(df_armonicos)
# Guardar la tabla como CSV
df_armonicos.to_csv('armonicos_pich.csv', index=False)
# Reconstrucción de la marea
tide = utide.reconstruct(t, coef, verbose=False)
# Only M2 K1 S2
m2_tide_data = utide.reconstruct(t, coef, verbose=False, constit = ['M2', 'K1', 'S2'])

   Armonico  Amplitud        Fase  Porcentaje de Energia
0        M2  0.435490   60.219877           7.083914e+01
1        K1  0.157724   41.973856           9.292023e+00
2        S2  0.149004   82.326926           8.293072e+00
3        O1  0.103614  356.125652           4.010107e+00
4        N2  0.098815   28.335329           3.647256e+00
..      ...       ...         ...                    ...
62      MS4  0.000122  233.413403           5.571899e-06
63     MSK6  0.000087  274.282566           2.832002e-06
64     3MK7  0.000079  286.763331           2.304618e-06
65      SN4  0.000063  167.953828           1.494317e-06
66       M8  0.000011  104.067500           4.674403e-08

[67 rows x 4 columns]


In [81]:
# Dataframe con datos de la reconstrucción
df_tide = pd.DataFrame({
    'Datos Originales Pichidangui': u,
    'Reconstruccion Pichidangui': tide.h,
    'M2 Pich': m2_tide_data.h,
    'Residuo Pichidangui': u-tide.h,       # Residuo entre la marea original y la marea gravitacional
    'fecha': t,
})

# Guardar los datos como CSV
df_tide.to_csv('reconstrucion_residuo_pich.csv', index=False)
# Cálculo de la proporción de variancias de los residuos y datos originales
print(df_tide['Residuo Pichidangui'].var()/df_tide['Datos Originales Pichidangui'].var())

0.0322586246617043


In [82]:
# Graficar los armónicos
#plt.xlabel('Fecha', fontsize=20) # Label de x
plt.ylabel('Nivel de la Marea [m]', fontsize=20) # Label de y
plt.plot(t, tide.h, color='navy', label='Nivel de la Marea [m]') # Ploteo de datos
plt.xticks(fontsize=18)                 
plt.yticks(fontsize=18) 

# Añadir leyenda
plt.tight_layout()
plt.legend(loc='upper right', fontsize=20)

# Mostrar el gráfico
plt.title('Marea Pichidangui (july 2023-september 2024)', fontsize=20) # titulo
plt.show() # Mostrar figura

In [83]:
# Graficar los residuos
#plt.xlabel('Fecha', fontsize=20) # Label de x
plt.ylabel('Residuo [m]', fontsize=20) # Label de y
plt.plot(t, u-tide.h, color='yellowgreen', label='Nivel de los Residuos [m]') # Ploteo de datos
plt.xticks(fontsize=18)                 
plt.yticks(fontsize=18) 

# Añadir leyenda
plt.tight_layout()
plt.legend(loc='upper right', fontsize=20)

# Mostrar el gráfico
plt.title('Residuos Pichidangui (july 2023-september 2024)', fontsize=20) # titulo
plt.show() # Mostrar figura

## Análisis de mareas altas y bajas 

In [85]:
# Primero calculamos la media movil para un rango de 30 min (suavizado de datos):
csv_filtered['Media movil 30min'] = csv_filtered['prs(m)'].rolling(30).mean()
# Exportamos un filtro para suavizar aun mas los datos
from scipy.signal import savgol_filter
Media_mov = np.split(np.array(csv_filtered['Media movil 30min']), [29]) # Vector de media movil
Smoot_Media_mov = savgol_filter(Media_mov[1], 500, 5)                   # Media movil suavizada
#Incorporamos la curva suavizada al dataframe
Smoot_Media_mov_joint = np.concatenate((Media_mov[0],Smoot_Media_mov), axis=0)
csv_filtered['Smooth Media Movil 30min'] = Smoot_Media_mov_joint

C:\Users\marib\AppData\Local\Temp\ipykernel_12888\108663410.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_filtered['Media movil 30min'] = csv_filtered['prs(m)'].rolling(30).mean()
C:\Users\marib\AppData\Local\Temp\ipykernel_12888\108663410.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv_filtered['Smooth Media Movil 30min'] = Smoot_Media_mov_joint


In [86]:
# Utilizamos análisis de señales en los datos suavizados
from scipy.signal import find_peaks

# Encontrar peaks (local maxima)
peaks, _ = find_peaks(csv_filtered['Smooth Media Movil 30min'], height=0)

# Encontrar valles local minima)
valleys, _ = find_peaks(-np.array(csv_filtered['Smooth Media Movil 30min']), height=None)

# Guardar los tiempos de los peaks y valles
peak_times = csv_filtered['fecha'].iloc[peaks].values
valley_times = csv_filtered['fecha'].iloc[valleys].values



### Debido a la media móvil los tiempos de máximos y mínimos se encuentran defasados. Procedemos a encontrar los máximos y mínimos reales.

In [95]:
#Convertir peak_times y valley_times en dataframes
df_peak_times = pd.DataFrame({
    'fecha': peak_times
})
df_valley_times = pd.DataFrame({
    'fecha': valley_times
})

# Parámetros del rango temporal 
time_window = pd.Timedelta(minutes=180)

# Función para encontrar máximos del rango temporal alrededor de cada peak y mínimo en valle
# df_numeric -> Datos originales
# peaks_t -> tiempos de los peaks suavizados
# window -> Ventana de tiempo donde buscar los verdaderos peaks
# inverted -> Busca valles
def find_max_in_window(df_numeric, peaks_t, window, inverted = False):
    # Filtrar el DataFrame numérico dentro del rango temporal
    mask = (df_numeric['fecha'] >= (peaks_t - window)) & (df_numeric['fecha'] <= (peaks_t + window))
    filtered_df = df_numeric[mask]
    
    # Encontrar el valor máximo y su fecha en datetime (lo mismo para valles)
    if not filtered_df.empty:
        if inverted == False:
            max_value = filtered_df['prs(m)'].max()
            max_time = filtered_df.loc[filtered_df['prs(m)'].idxmax(), 'fecha']
            return max_value, max_time
        else:
            min_value = filtered_df['prs(m)'].min()
            min_time = filtered_df.loc[filtered_df['prs(m)'].idxmin(), 'fecha']
            return min_value, min_time
    else:
        return None, None

# Aplicar la función a cada peak del DataFrame 1
results_peaks = []
for index, row in df_peak_times.iterrows():
    timestamp = row['fecha']
    max_value, max_time = find_max_in_window(csv_filtered, timestamp, time_window)
    results_peaks.append({'HT peak (m)': max_value, 'real peak time': max_time})

# Crear un DataFrame con los resultados
df_results_HT = pd.DataFrame(results_peaks)

# Aplicar la función a cada peak del DataFrame 1
results_valleys = []
for index, row in df_valley_times.iterrows():
    timestamp = row['fecha']
    min_value, min_time = find_max_in_window(csv_filtered, timestamp, time_window, inverted = True)
    results_valleys.append({'LT valley (m)': min_value, 'real valley time': min_time})

# Crear un DataFrame con los resultados
df_results_LT = pd.DataFrame(results_valleys)

In [163]:
#Comprobamos si los tiempos de los peaks coinciden con los datos reales
plt.plot(csv_filtered['fecha'], csv_filtered['prs(m)'], label= 'NM [m]', color = 'black', linewidth = 2)
plt.ylabel('Nivel del Mar [m]', fontsize=24) # Label de y
plt.xticks(fontsize=18)                 
plt.yticks(fontsize=22) 
plt.scatter(df_results_HT['real peak time'], df_results_HT['HT peak (m)'], label='Marea Alta', color = 'yellowgreen', linewidths = 8)
plt.scatter(df_results_LT['real valley time'], df_results_LT['LT valley (m)'], label='Marea Baja', color = 'slateblue', linewidths = 8)
plt.legend(fontsize = 20)
plt.show()

In [169]:
#Plot de High and Low tide
plt.ylabel('Nivel del Mar [m]', fontsize=24) # Label de y
plt.xticks(fontsize=18)                 
plt.yticks(fontsize=22) 
plt.plot(df_results_HT['real peak time'], df_results_HT['HT peak (m)'], label='Marea Alta', color = 'yellowgreen', linewidth = 3)
plt.plot(df_results_LT['real valley time'], df_results_LT['LT valley (m)'], label='Marea Baja', color = 'slateblue', linewidth = 3)
plt.legend(fontsize = 20)
plt.show()